This notebook is to build a tashkeel model for arabic poems, it is based on 14 sequence length (best tell now based on our expirements) 
it utilized Bidirectional LSTM network for this task.
first we clean training data then we separate the letter from diacritics in two list.
then we build the sequences and prepare the classes
after that train the model
at the end you canfind the testing model code.

In [2]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model
from tashkeel_lib import basic_diacritics, diacritics , full_diacritics ,clean_text,clean_bayt

Load training data

In [3]:
raw_text = open("main_6_Bayt.csv", 'r', encoding='utf-8').read()

clean raw data from non punctuations

In [4]:
raw_text = clean_text( raw_text)

build dictionary for class to integer conversion and integer to class

In [5]:
diac_to_int = dict((c, i) for i, c in enumerate(full_diacritics))
int_to_diac = dict((i, c) for i, c in enumerate(full_diacritics))

Build two lists one for letter and one for diacritics 

In [6]:
text = []
tashkeel = []

prev_char = ' '
text.append(prev_char)

text_index = 0
while text_index < len(raw_text):
     cur_char = raw_text[text_index] 
     if cur_char not in diacritics and  prev_char not in diacritics: # letterv then letter
        text.append(cur_char)
        tashkeel.append('z')
        prev_char= cur_char
     elif cur_char not in diacritics and  prev_char in diacritics: # tashkeel then letter
        text.append(cur_char)
        prev_char= cur_char
     elif cur_char == 'ّ' and prev_char in basic_diacritics :  # diacritics before chaddah
        print('Data error diacritics before chaddah ' + 'ح' + prev_char)
        break
     elif cur_char in basic_diacritics and prev_char in basic_diacritics :  # double diacritics
        print('Data error double diacritics' + 'ح' + cur_char +'   '+ 'ح' + prev_char)
        break
     elif cur_char in basic_diacritics :
        tashkeel.append(cur_char)
        prev_char = cur_char 
     elif cur_char == 'ّ':  # shaddah
        if raw_text[text_index +1 ]  in basic_diacritics:
            tashkeel.append(raw_text[text_index ]+ raw_text[text_index +1 ])
            prev_char = raw_text[text_index +1 ]
            text_index = text_index +1
        else:
            tashkeel.append(cur_char)
            prev_char = cur_char
     else:
        print('Data error uncovered case')
        break
     text_index = text_index+1

In [8]:
len(text)

10915086

regenerating original text to test the previous step (separation)

In [9]:
for i in range (200):
    print(text[i] + tashkeel[i].replace('z',''), end='')

 وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَستَوجِبا مَنّاً عَلَيَّ وَتُحمَدا 
 وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفرَعَ فِي لَومي مِراراً وَأَصعَدا 
 وَإِن صَرَّحَت كَحلٌ وَهَبَّت عَرِيَّةٌ مِنَ الريحِ لَمْ تَترُك لِذي المالِ مِرفَدا 
 وَلم يَحمِ فَرجَ الحَيِّ إِلَّا مُحافِظٌ كَريمُ المُ

Dictionary to convert from letter class to int and back

In [10]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


# summarize the loaded data
n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  10915086
Total Vocab:  38


In [11]:
seq_length = 14

Building sequences of letters and the prediction diacritics

In [12]:
# prepare the dataset of input to output pairs encoded as integers
n_chars = len(tashkeel)
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = text[i:i + seq_length -7 ]+ text[i + seq_length -7: i + seq_length ]
	seq_out = tashkeel[i + seq_length-7 ]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(diac_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)


Total Patterns:  10915071


treating madd letters sequences

In [13]:
keepX = []
keepY = []
for i , t in  enumerate (dataY):
    if t >= 1:
        keepX.append(dataX[i]) 
        keepY.append(dataY[i])
    elif t == 0 and int_to_char[dataX[i][7]] in ['ا', 'آ', 'و', 'ي', 'ى','ل']:
        keepX.append(dataX[i]) 
        keepY.append(0)
len(keepY)

7248516

In [14]:
# clear memory
dataY =[]
dataX =[]
raw_text = ' '

In [15]:
# reshape X to be [samples, time steps, features]
n_patterns = len(keepX )
X = numpy.reshape(keepX, (n_patterns, seq_length, 1))


In [16]:
# normalize
X = X / float(n_vocab)


In [17]:
# one hot encode the output variable
y = np_utils.to_categorical(keepY)


In [20]:
# two layers LSTM 
stopped for future expirements
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))


define the models

In [51]:
model = Sequential()
model.add(Bidirectional(LSTM(256, input_shape=(X.shape[1], X.shape[2]))))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [52]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Start training then save the resulting model

In [53]:
s =model.fit(X, y, epochs=6, batch_size=128, callbacks=callbacks_list)
model.save('tashkeel_22_5__M6__7_7.h5')

Epoch 1/6
56630/56630 [==============================] - 2792s 49ms/step - loss: 0.7923

Epoch 00001: loss improved from inf to 0.79234, saving model to weights-improvement-01-0.7923.hdf5
Epoch 2/6
56630/56630 [==============================] - 2730s 48ms/step - loss: 0.5915

Epoch 00002: loss improved from 0.79234 to 0.59147, saving model to weights-improvement-02-0.5915.hdf5
Epoch 3/6
56630/56630 [==============================] - 2624s 46ms/step - loss: 0.5422

Epoch 00003: loss improved from 0.59147 to 0.54225, saving model to weights-improvement-03-0.5422.hdf5
Epoch 4/6
56630/56630 [==============================] - 2604s 46ms/step - loss: 0.5146

Epoch 00004: loss improved from 0.54225 to 0.51458, saving model to weights-improvement-04-0.5146.hdf5
Epoch 5/6
56630/56630 [==============================] - 2636s 47ms/step - loss: 0.4962

Epoch 00005: loss improved from 0.51458 to 0.49618, saving model to weights-improvement-05-0.4962.hdf5
Epoch 6/6
56630/56630 [=====================

Load a pretrained model or skip to use current trained model

In [18]:
model = load_model('tashkeel_22_5__M6__7_7.h5')

testing the model: note the input is test file cotains unvocalized poems "test_bayts.txt" and the result is wretten to "test_bayt_results.txt"

In [19]:
test_file = open ('test_bayts.txt', 'r', encoding='utf-8')
test_bayts = test_file.read()
test_file.close()
test_bayts = test_bayts.split('\n')

result_file = open('test_bayt_results.txt', 'w', encoding='utf-8')

for bayt in test_bayts:
    bayt =  clean_bayt(bayt)
    seq = []
    for i in range (seq_length -9 ):
      seq.append(1)  # space
    seq.append(0)    # \n
    seq.append(1)   # space
    
    for i in range(7) :
       seq.append(char_to_int[bayt[i]])
    
    # reset variables 
    bayt_len = len(bayt)
    curr_pos = 0
    bayt_tashkeel =[]
    
    while curr_pos <= bayt_len -8 :
        bayt_tashkeel.append( bayt[curr_pos])
        x = numpy.reshape(seq, (1, len(seq), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        if bayt[curr_pos] not in [' ','\n'] and index !=0:
           bayt_tashkeel.append( int_to_diac[index])

        seq = seq[1:]
        seq.append(char_to_int[bayt[curr_pos +7]])
        curr_pos = curr_pos +1
    
    result_file.write(''.join(bayt_tashkeel).strip() +'\n')
result_file.close()

In [ ]:
show last bayt result ( if you cannt wait to open the file :)

In [21]:
''.join(bayt_tashkeel)

'يَحْكُمُ فِيهِ رَشَدَهُ فَهْوَ غَانِمٌ وَمَنْ لَمْ يَحِكُمُ يَشْرَبُ فَهْوَ غَارِمِ\n     '